In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [22]:
def extract_features(path, label, data, labels):
  videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
  for videoPath in videoPaths:
    cap = cv2.VideoCapture(videoPath)

    ret, frame = cap.read()

    frame = cv2.resize(frame, (160, 120))


    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()

    mask = np.zeros_like(frame)
    # Sets image saturation to maximum
    mask[..., 1] = 255

    times = 0

    while(cap.isOpened()):



        ret, curr_frame = cap.read()



        times += 1
        if times != 28:
          continue  

        times = 0
        if not ret:
          break

        curr_frame = cv2.resize(curr_frame, (160, 120)) 


        gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        flow = optical_flow.calc(prev_gray, gray, None)


        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Sets image hue according to the optical flow direction
        mask[..., 0] = angle * 180 / np.pi / 2
        # Sets image value according to the optical flow magnitude (normalized)
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

        h, s, image = cv2.split(mask)

        
        kernel = np.array([[-1,-1,-1], 
                          [-1, 9,-1],
                          [-1,-1,-1]])
        image = cv2.filter2D(image, -1, kernel)

        image = cv2.Canny(image,100,200)


        kernel = np.ones((5,5),np.uint8)
        image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)


        image = cv2.morphologyEx(image,cv2.MORPH_OPEN,kernel)

        
        hog = cv2.HOGDescriptor()
        image = hog.compute(image)

        # Updates previous frame
        prev_gray = gray

        data.append(image)
        labels.append(label)
    print(videoPath + "---done--")



In [23]:
labels = []
data = []

In [ ]:
path = "/content/drive/My Drive/Data/cobaoluc"
extract_features(path, 0, data, labels)
path = "/content/drive/My Drive/Data/khongbaoluc"
extract_features(path, 1, data, labels)



/content/drive/My Drive/Data/cobaoluc/0.mp4---done--


In [1]:
print(len(data))
print(len(labels))

NameError: ignored

In [ ]:
trainX, testX, trainY, testY = train_test_split(data, labels,
	test_size=0.20, random_state=42)

In [ ]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(features_train, labels_train))
print(lsvc.score(features_test, labels_test))
